In [54]:
import numpy as np

In [55]:
def z_score_normalization(x_features):
    m_mean = np.mean(x_features, axis=0)
    s_std = np.std(x_features, axis=0)
    normalized_features = (x_features - m_mean) / s_std
    return normalized_features

In [56]:
def compute_cost(x, y, w, b, l):
    predictions = np.dot(x, w) + b
    errors = predictions - y
    cost_result = np.mean(errors ** 2) / 2
    regularization_cost = (l / (2 * len(y))) * np.sum(np.square(w))
    return cost_result + regularization_cost

In [57]:
def gradient_descent(x, y, w, b, lr, ni, l):
    m = len(y)
    lh = []

    for iteration in range(ni):
        # 计算预测值
        predictions = np.dot(x, w) + b

        # 计算误差
        errors = predictions - y

        # 计算梯度
        dw = np.dot(x.T, errors) / m
        db = np.sum(errors) / m

        # 计算正则化梯度
        dw_reg = dw + (l / m) * w

        # 更新参数
        w -= lr * dw_reg
        b -= lr * db

        # 计算和记录成本
        cost = compute_cost(x, y, w, b, l)
        lh.append(cost)

        # 输出调试信息
        if iteration % 1000 == 0:
            print(f"Iteration {iteration}: Cost {cost:.2f}, "
                  f"w {[f'{val:.2f}' for val in w]}, b {b:.2f}")

    return w, b, lh

In [58]:
def train_logistic_regression(x, y, lr=0.01, ni=2000, l=0.01):
    # 初始化权重和偏置项
    w = np.zeros(features.shape[1])
    b = 0.0

    # 执行梯度下降
    w, b, lh = gradient_descent(x, y, w, b, lr, ni, l)

    return w, b, lh

In [59]:
from sklearn.preprocessing import PolynomialFeatures
features = np.array([[32.92519127, 27.83917713],
                     [27.11848509, 77.13174390],
                     [13.11572466, 73.54715439],
                     [14.94266746, 59.83553954],
                     [26.60643383, 72.74404041],
                     [32.28633520, 40.89511783],
                     [38.40147704, 29.27157782],
                     [35.32576878, 50.48961658],
                     [33.87473740, 53.16308674],
                     [12.58521539, 64.25175499]])

target = np.array([ 175.81028952, 114.98247684, 113.13922922, 139.89057231, 152.51222144,
                    165.20163529, 161.78325149, 153.59759730, 169.31041003, 142.59595240 ])

# Z-score 标准化特征
features_normalized = z_score_normalization(features)

# 多项式特征变换
poly = PolynomialFeatures(degree=3, include_bias=False)
features_poly = poly.fit_transform(features_normalized)

In [60]:
import plotly.graph_objs as go
# 创建3D图形
def create_surface_plot(w, b, title):
    # 生成网格
    x = np.linspace(features[:, 0].min(), features[:, 0].max(), 50)
    y = np.linspace(features[:, 1].min(), features[:, 1].max(), 50)
    x_grid, y_grid = np.meshgrid(x, y)

    # 将网格应用归一化
    grid_features = np.c_[x_grid.ravel(), y_grid.ravel()]
    grid_features_normalized = z_score_normalization(grid_features)

    # 将网格应用多项式变换
    grid_poly = poly.transform(grid_features_normalized)

    # 计算预测值
    z_grid = np.dot(grid_poly, w) + b
    z_grid = z_grid.reshape(x_grid.shape)

    # 创建图形
    surface = go.Surface(x=x_grid, y=y_grid, z=z_grid, colorscale='Viridis', opacity=0.7)
    scatter = go.Scatter3d(x=features[:, 0], y=features[:, 1], z=target, mode='markers', marker=dict(size=5, color='red'))

    layout = go.Layout(title=title, scene=dict(xaxis_title='Temperature', yaxis_title='Humidity', zaxis_title='Ice Cream Sales'))
    fig = go.Figure(data=[surface, scatter], layout=layout)
    return fig

In [61]:
# 训练模型，分别迭代100次和1000次
w_1000, b_1000, _ = gradient_descent(features_poly, target, np.zeros(features_poly.shape[1]), 0.0, lr=0.01, ni=1000, l=0.001)
w_10000, b_10000, _ = gradient_descent(features_poly, target, np.zeros(features_poly.shape[1]), 0.0, lr=0.01, ni=10000, l=0.001)
w_50000, b_50000, _ = gradient_descent(features_poly, target, np.zeros(features_poly.shape[1]), 0.0, lr=0.01, ni=50000, l=0.001)

Iteration 0: Cost 10440.62, w ['0.14', '-0.17', '1.44', '-0.98', '1.50', '-0.54', '-0.09', '0.57', '-0.76'], b 1.49
Iteration 0: Cost 10440.62, w ['0.14', '-0.17', '1.44', '-0.98', '1.50', '-0.54', '-0.09', '0.57', '-0.76'], b 1.49
Iteration 1000: Cost 88.81, w ['35.52', '-8.79', '24.05', '27.60', '11.64', '-12.07', '1.60', '0.87', '-1.07'], b 122.05
Iteration 2000: Cost 51.72, w ['33.51', '-7.19', '17.08', '32.26', '7.69', '-14.18', '0.24', '-0.46', '-5.42'], b 135.90
Iteration 3000: Cost 43.14, w ['30.61', '-4.57', '13.14', '29.53', '4.96', '-13.53', '-1.41', '-2.02', '-7.60'], b 141.23
Iteration 4000: Cost 36.84, w ['28.14', '-2.21', '9.82', '25.85', '2.58', '-12.77', '-2.94', '-3.42', '-9.17'], b 144.92
Iteration 5000: Cost 31.78, w ['26.16', '-0.15', '6.76', '22.20', '0.41', '-12.26', '-4.37', '-4.56', '-10.42'], b 148.04
Iteration 6000: Cost 27.65, w ['24.60', '1.67', '3.88', '18.74', '-1.60', '-11.99', '-5.68', '-5.48', '-11.46'], b 150.79
Iteration 7000: Cost 24.23, w ['23.39',

In [62]:
# 绘制在迭代100次的模型
fig_1000 = create_surface_plot(w_1000, b_1000, '3D Regression Surface (1000 Iterations)')
fig_1000.show()

In [63]:
# 绘制在迭代1000次的模型
fig_10000 = create_surface_plot(w_10000, b_10000, '3D Regression Surface (10000 Iterations)')
fig_10000.show()

In [64]:
fig_50000 = create_surface_plot(w_50000, b_50000, '3D Regression Surface (50000 Iterations)')
fig_50000.show()